How does GPT-2 behaves in summarization tasks?

In this notebook we'll be following the summarization experiment mentioned in the [GPT-2 paper](https://d4mucfpksywv.cloudfront.net/better-language-models/language-models.pdf).

We'll be using some articles from the [cnn_daily mail dataset](https://huggingface.co/datasets/cnn_dailymail) at HuggingFace, and the larger GTP-2 model.

Quoting from the GPT-2 paper, we'll be doing this:

```
To induce summarization behavior we add the text TL;DR: after
the article and generate 100 tokens with Top-k random sam-
pling (Fan et al., 2018) with k = 2 which reduces repetition
and encourages more abstractive summaries than greedy de-
coding. We use the first 3 generated sentences in these 100
tokens as the summary.
```

In [1]:
import random
from gptbench import Sample, empty_config

In [2]:
ben = Sample(seed=0xDEEDD0D0)

cfg = empty_config()

cfg.model.set(dtype='bfloat16')

# the next sample config settings are important:
# max_len=100 - generate 100 tokens
# top=2 will only emit the two most probable tokens on each step (top_k=2)
# emit_start=False will skip emitting the initial context
cfg.sample.set(max_len=100, top=2, emit_start=False)

# if you get an out of memory error, try 'gpt2', the smaller model:
ben.init_pretrained('gpt2-xl', cfg)

Initializing model from gpt2-xl
Dataset: dummy 0 tokens
Dataset: loading uint16 tokens
Expanding initial dataset size of 1 (less than block_size+1) by 1025 times to size of 1025
Dataset train_path: dummy empty dataset, val_path: None, train_split: 0.9, vocab_size: 50257
Model params: 1557.61M


In [3]:
# a function to return the first n paragraphs
def first_paragraphs(text, count=3):
    s = text.split('.')
    return '.'.join(s[:count]) + '.'

In [4]:
# an article about Harry Potter's star
article="""LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details of how he'll mark his landmark birthday are under wraps. His agent and publicist had no comment on his plans. "I'll definitely have some sort of party," he said in an interview. "Hopefully none of you will be reading about it." Radcliffe's earnings from the first five Potter films have been held in a trust fund which he has not been able to touch. Despite his growing fame and riches, the actor says he is keeping his feet firmly on the ground. "People are always looking to say 'kid star goes off the rails,'" he told reporters last month. "But I try very hard not to go that way because it would be too easy for them." His latest outing as the boy wizard in "Harry Potter and the Order of the Phoenix" is breaking records on both sides of the Atlantic and he will reprise the role in the last two films. Watch I-Reporter give her review of Potter's latest » . There is life beyond Potter, however. The Londoner has filmed a TV movie called "My Boy Jack," about author Rudyard Kipling and his son, due for release later this year. He will also appear in "December Boys," an Australian film about four boys who escape an orphanage. Earlier this year, he made his stage debut playing a tortured teenager in Peter Shaffer's "Equus." Meanwhile, he is braced for even closer media scrutiny now that he's legally an adult: "I just think I'm going to be more sort of fair game," he told Reuters. E-mail to a friend . Copyright 2007 Reuters. All rights reserved.This material may not be published, broadcast, rewritten, or redistributed."""
start_text = article + '\n' + 'TL;DR' + '\n'

# we're sampling with the sample config settings defined above: max_len=100, top=2, emit_start=False
out=[]
ben.sample(start_text, dest=out)
first_paragraphs(out[0]) 

"Daniel Radcliffe is 18 years old. He's a famous actor. He's not going to spend his life buying expensive cars."

In [5]:
out=[]
ben.sample(start_text, dest=out)
first_paragraphs(out[0]) 

"Harry Potter star Daniel Radcliffe is 18 and has a reported £20 million fortune. He's not planning on frittering it away.\nDaniel Radcliffe is 18 and has a reported £20 million fortune."

The human-generated summary:
```
Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday . Young actor says he has no plans to fritter his cash away . Radcliffe's earnings from first five Potter films have been held in trust fund .
```

In [9]:
# another article about obesity
article = """NEW YORK (CNN) -- Obesity is the No. 1 health crisis in the United States, and the nation could be at risk without immediate action, former President Clinton said at CNN's first Fit Nation Summit. CNN's Dr. Sanjay Gupta and former President Clinton lead the discussion at the first Fit Nation Summit. "We need to do something about it for our children, and for our country, because something like this could easily collapse our nation if we don't act now," Clinton said. Clinton and CNN's chief medical correspondent, Dr. Sanjay Gupta, gathered with experts from across the country Wednesday to explore solutions to America's obesity crisis. Two-thirds of adult Americans are overweight or obese; the same is true for one-third of U.S. children, according to the Centers for Disease Control and Prevention. Obesity carries increased risk of myriad health problems, including hypertension, diabetes, heart disease and arthritis. Gupta, Clinton and a panel of experts addressed such proposed solutions as healthy school lunches, elimination of trans fats, the need for healthier food in urban communities and the importance of supporting community-based efforts to fight obesity. Watch Gupta explain the solutions proposed at the Fit Nation Summit » . Clinton, who's been tackling the obesity issue over the past few years with his Alliance for a Healthier Generation, stressed the need to help educate people about the importance of a healthy diet and exercising together. Motivating Americans, especially kids, won't be easy, Clinton said. "They need to be handled with care. Kids need to know it's important, but [fitness] can't be boring," he said. "There's no shame, there's no embarrassment. We have to let them know it's all good, but you have to do it." Michael Jacobson, director of the Center for Science in the Public Interest and a strong advocate against trans fats in foods, stressed that the Food and Drug Administration should ban trans fats altogether and stop leaving it up to local and state governments. Trans fats, at one time, thought to be good fat substitutes, over the years have been shown to raise LDL or bad cholesterol in people and lower their HDL, or good cholesterol. "They're just not good for our health, Jacobson said. However, Dr. Robert Eckel, former president of the American Heart Association, warned, "Although we support the removal of trans fats, our biggest concern is what they'll be replaced with. We need to think about that, because there will be a substitute." The panel took questions from an audience of 150 people -- mostly obesity experts or those who work in industries that deal with obesity. Gupta was inspired to launch Fit Nation, an ongoing, multiplatform, grass-roots initiative against obesity. Over the past three years, Gupta and the CNN Medical Unit have crisscrossed the country, asking Americans to take charge of their weight by exercising more and eating healthier. This year, Gupta traveled from Michigan to California asking the public to pledge to add 1 million hours to their collective lives by getting off the couch and exercising. Since he began the tour in April, the campaign has surpassed its goal, with exercise pledges close to four times that amount. Gupta said he hopes that legislators and administrators who can make a difference in policy and perhaps change attitudes about obesity will hear the solutions discussed at the summit. "I''ve covered a lot of stories on obesity," Gupta said. "There's a battle going on, and we need to win the battle. I've talked to people out there and many are just blasé. "This is our opportunity to make them care." E-mail to a friend ."""
start_text = article + '\n' + 'TL;DR' + '\n'

print(f"Prompt is {len(ben.train_dataset.encode(article))} tokens (up to {ben.model.block_size})")

out=[]
ben.sample(start_text, dest=out)
first_paragraphs(out[0]) 

Prompt is 738 tokens (up to 1024)


'1. Obesity is the No. 1 health crisis in the United States.'

In [10]:
out=[]
ben.sample(start_text, dest=out)
first_paragraphs(out[0])

'The first Fit Nation Summit was a great success. It was an opportunity to hear the solutions proposed by experts from across the country. The summit was also a chance to hear from the experts who are working on solutions.'

The human-generated summary:
```
Experts gather at CNN's first Fit Nation Summit to propose obesity solutions . Former President Clinton: Without obesity solution, nation risks "collapse" Motivating youngsters, removing shame, embarrassment called key . Experts: Removing trans fats, finding healthy replacements also critical .
```

In [11]:
# an article about a painting exhibition
article = """ATLANTA, Georgia (CNN) -- It is an irony of contemporary art-museum management: Sometimes the museum that creates an exhibition doesn't get to premiere it. In a treatment of a Bridgeman Art Library photo, the High shows visitors Jules Arnout's "View of the Grand Gallery." This is the case this week, as Atlanta's High Museum of Art couples its public opening of a second year of Louvre-fueled shows from Paris, France, with an exhibition about influences on the Impressionists. "Inspiring Impressionism" is organized by the Denver Art Museum. It opened Tuesday at the High, to run there through January 2008. With the backing of Northern Trust, the show will then travel to Denver February 23 to May 25, 2008, and then on to the Seattle Art Museum from June 19 to September 21, 2008. Why not start in Denver? That museum this fall is host to pieces seen in the first year of the three-year Louvre Atlanta series of exhibitions. Like priceless dominos falling, these shows roll around the country and the world, globalization dictating galas, super-sensitive custom shipping companies probably among the biggest winners. See a gallery of images from 'Inspiring Impressionism' at the High Museum of Art » . Another unintended effect may be shadowing of one important outing by another. It should be interesting to learn whether showing "Inspiring Impressionism" at the High on the same time frame as the second year of Louvre Atlanta pays off. Do all the boats in Monet's "Autumn on the Seine, Argenteuil" float? Smart museum-goers will see it all. "The Louvre and the Ancient World" and the companion show "The Eye of Josephine," after all, look at some of the oldest treasures in the Louvre's vast holdings. See a gallery of some of the highlights of the Louvre Atlanta shows » . The fit is comfortable, in a way, the Impressionist movement of the turn of the last century finding its main proponents in French artists. The emphasis here, however, is on what older works may have moved and motivated such artists as Claude Monet, Pierre-Auguste Renoir and Paul Cézanne. One very distinctive connection from the Louvre Atlanta opener of last year is a section of this new show that looks at the tradition of modern-era artists learning by studying the work of masters at the Louvre, Bartolomé Esteban Murillo's "The Beggar Boy" of around 1650 being one of the most-copied canvases in Western art's modern history. From the outset, visitors to "Inspiring Impressionism" are reminded of the Louvre treasures all around them at the High: The Impressionism show's entry gallery is flanked by a handsome photographic treatment of Louis Jules Arnout's "View of the Grand Gallery at the Louvre" from between 1850 and 1870. In that original painting and color lithograph that followed, Arnout captured the bustle of artists working and visitors promenading in the Louvre's chief exhibition space that runs along the Seine in Paris. In fact, even older-era echoes of this same concept are encountered on looking at Louvre Atlanta's "The Tiber" marble from the first century A.D., you're reminded that Michelangelo himself was aware of that piece, influenced by it, presumably inspired by it. So a surprise symmetry takes shape at the High this fall, as visitors contemplate dialogues between museums (the Louvre, the High, Denver, Seattle), the viewers of art in Europe and the United States, and the artists themselves in France, in the U.S. and elsewhere. High director Michael Shapiro calls these synchronicities "visual evidence of connections." His colleagues Timothy Standring of Denver and Ann Dumas of London, England, have held up their end of this conversational eyeful with timely contributions, glimpses of Old World craft from Titian and Velasquez to Fragonard and Rubens -- and the "moderns" who saw beyond them to a new age of aesthetic debate. Don't miss some of the comments and writings of various artists, used as part of the display of the show at the High. Degas may have said it most honestly: "No art is less spontaneous than mine. What I do is the result of refection and the study of the Old Masters." Joining Northern trust Corporation in funding the exhibition are the National Endowment for the Humanities, and support is also provided by an indemnity from the Federal Council on the Arts and the Humanities. E-mail to a friend ."""
start_text = article + '\n' + 'TL;DR' + '\n'

print(f"Prompt is {len(ben.train_dataset.encode(article))} tokens (up to {ben.model.block_size})")

out=[]
ben.sample(start_text, dest=out)
first_paragraphs(out[0]) 

Prompt is 956 tokens (up to 1024)


'The Louvre Atlanta exhibition "Inspiring Impressionism," which opened Tuesday, February 3, 2008, at the High Museum of Art in Atlanta, features works by artists including Claude Monet, Paul Cézanne, Henri Matisse and Pablo Picasso. The show is organized by the Denver Art Museum. See the High\'s "Inspiring Impressionism" exhibit » .'

In [12]:
out=[]
ben.sample(start_text, dest=out)
first_paragraphs(out[0])

'The High Museum of Art in Atlanta, Georgia, has a new show on Impressionism. The show is called "Inspiring Impressionism" and runs from February 23 to May 25, 2008. The show is on view at the High through January 8, 2009.'

For comparision, here's the human summary:
```
"Inspiring Impressionism" looks at Old Masters, other influences on modern art . Featured are works of Monet, Renoir, Cézanne, Cassatt, Morisot . "Inspiring Impressionism" runs through January 31, 2008, at the High Museum . Denver Art Museum and Seattle Art Museum get the show next .
```

Variable results but mostly bad.

Perhaps fine-tunning with the cnn_daily_mail dataset could teach GPT-2 summarization skills?